In [8]:
from typing import Literal
from tqdm import tqdm
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    load_chat_planner, load_agent_executor, PlanAndExecute
)

import pandas as pd

In [9]:
# load CSV
csv_dir = "../target_csv/"
csv_name = "train_data_smi_unique_target_name"
df = pd.read_csv(csv_dir + csv_name +".csv")

In [10]:
ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategies = "zero-shot-react"
) -> LLMChain:
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

In [11]:
# 에이전트 로드
tool_names = ["wikipedia",'pubmed']
agent_chain = load_agent(tool_names, strategy="plan-and-solve")

In [12]:
# Description 컬럼을 생성하여 각 protein_name에 대한 설명을 추가 및 저장
if "description" not in df.columns:
    df["description"] = ""

In [13]:
save_threshold = 10  # 열 개마다 저장
save_count = 0

In [15]:
for idx, protein_name in tqdm(enumerate(df["Target Name"]), desc="Fetching descriptions", total=len(df)):
    if not df.at[idx, "description"]:  # 이미 설명이 채워진 경우 건너뜀
        query = f"Please provide a brief description of {protein_name}."
        try:
            description = agent_chain.invoke(query, handle_parsing_errors=True)
        except ValueError as e:
            description = f"Error: {str(e)}"
        
        df.at[idx, "description"] = description
        save_count += 1
        hex_count = 'A'

        # 열 개의 설명이 추가되면 저장
        if save_count >= save_threshold:
            df.to_csv(csv_name +" cnt,"+ hex_count + save_count + ".csv", index=False)
            save_count = 0  # 저장 후 카운트 초기화
            hex_count += 1

Fetching descriptions:   0%|          | 0/61 [00:00<?, ?it/s]



> Entering new PlanAndExecute chain...
steps=[Step(value='Define what Tyrosine-protein kinase receptor RET is, including its classification and general function.'), Step(value='Describe the role of RET in cellular signaling and its importance in development and physiology.'), Step(value='Mention any known associations with diseases or conditions related to RET.'), Step(value="Summarize the key points in a concise manner.\nGiven the above steps taken, please respond to the user's original question. \n")]

> Entering new AgentExecutor chain...
Thought: I need to gather information about Tyrosine-protein kinase receptor RET, including its classification and general function. This will likely involve looking up relevant scientific and biological information. 

Action:
```
{
  "action": "wikipedia",
  "action_input": "Tyrosine-protein kinase receptor RET"
}
```

Observation: Page: Receptor tyrosine kinase
Summary: Receptor tyrosine kinases (RTKs) are the high-affinity cell surface recepto

c:\Users\stu83\anaconda3\envs\dacon\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\stu83\anaconda3\envs\dacon\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: RET proto-oncogene
Summary: The RET proto-oncogene encodes a receptor tyrosine kinase for members of the glial cell line-derived neurotrophic factor (GDNF) family of extracellular signalling molecules.
RET loss of function mutations are associated with the development of Hirschsprung's disease, while gain of function mutations are associated with the development of various types of human cancer, including medullary thyroid carcinoma, multiple endocrine neoplasias type 2A and 2B, pheochromocytoma and parathyroid hyperplasia.

Page: Hirschsprung's disease
Summary: Hirschsprung's disease (HD or HSCR) is a birth defect in which nerves are missing from parts of the intestine. The most prominent symptom is constipation. Other symptoms may include vomiting, abdominal pain, diarrhea and slow growth. Most children develop signs and symptoms shortly after birth. However, others may be diagnosed later in infancy or early childhood. About half of all children with Hirschsprung'

Fetching descriptions:   2%|▏         | 1/61 [02:51<2:51:01, 171.03s/it]

...

Fetching descriptions:  15%|█▍        | 9/61 [09:54<35:47, 41.29s/it]

Thought: I need to summarize the importance of JAK2 in medical research and its potential as a therapeutic target based on the previous steps taken.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "JAK2 (Janus kinase 2) is a crucial non-receptor tyrosine kinase involved in signaling pathways for various cytokine receptors, playing a significant role in hematopoiesis and immune response. Its dysregulation, particularly through mutations like JAK2 V617F, is associated with myeloproliferative neoplasms (MPNs) such as essential thrombocythemia and polycythemia vera. This makes JAK2 a significant target for therapeutic intervention, as inhibiting its activity can help manage these conditions and potentially prevent the progression to acute myeloid leukemia (AML). JAK2's role in cell proliferation and differentiation underscores its importance in medical research, particularly in developing targeted therapies for hematological malignancies."
}
```


> Finished chain.
*****



Fetching descriptions:  15%|█▍        | 9/61 [10:43<1:01:58, 71.51s/it]

Thought: I need to summarize the information provided in the previous steps to respond to the user's original question effectively. The summary should include key points about each protein or receptor discussed, highlighting their functions, roles in diseases, and significance in health.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "The Tyrosine-protein kinase receptor RET is essential for signaling in the GDNF family, crucial for the development of the enteric nervous system and gastrointestinal motility, with mutations linked to Hirschsprung disease and various cancers. The Macrophage-stimulating protein (MSP) receptor, also known as RON, regulates inflammation and immune responses, playing a role in chronic inflammation, autoimmune diseases, and potentially facilitating immune escape in tumors. Cyclin-dependent kinase 7 (CDK7) is a key regulator of the cell cycle and transcription, implicated in various cancers and emerging as a promising therapeutic target. Seri

TypeError: can only concatenate str (not "int") to str

In [ ]:
# 모든 작업이 끝난 후 마지막으로 저장 (남아있는 데이터 저장)
if save_count > 0:
    df.to_csv(csv_name + "_descriptions.csv", index=False)

In [ ]:
# # 예시 도구 이름 리스트
# tool_names = ["wikipedia"]

# # 에이전트 로드 (제로샷 모델 사용)
# agent_chain = load_agent(tool_names, strategy="zero-shot-react")

# # 에이전트 실행
# result = agent_chain.run("Explain about T1-Faker")

# print("Result:\n", result)